In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix, classification_report
from keras.wrappers.scikit_learn import KerasClassifier
from keras.models import Sequential
from keras.utils import to_categorical, np_utils
from keras.layers import Dense, Dropout
import eli5
from eli5.sklearn import PermutationImportance

np.random.seed(42)
dir_path = '../2. Prepared Data/'
save_path = dir_path
file_name = ['20180924_merged_train_sampled_simple.csv','20180924_merged_test_sampled_simple.csv',\
             '20180925_undersampled_train_plus_test']

## <u> Keras NN</u>:

In [22]:
#TODOS OS DADOS
df = pd.read_csv(dir_path+file_name[0]).append(pd.read_csv(dir_path+file_name[1]))

X_df = df.drop('TIME',axis=1)
y_df = df.TIME

classes = list(np.unique(y_df))
n_classes = len(classes)
cols = list(X_df.columns)
n_cols = len(cols)

X_data,y_data = X_df.values.astype(float), y_df.values

#CODIFICANDO VETOR DE CLASSES PARA MATRIZ BINARIA 
encoder = LabelEncoder()
encoder.fit(y_data)
encoded_y = encoder.transform(y_data)
dummy_y = np_utils.to_categorical(encoded_y)

In [38]:
#CONSTRUCAO DO MODELO 
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(512, input_dim = n_cols, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(512,activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(29,activation = 'softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

estimator = KerasClassifier(build_fn=baseline_model, epochs=12, batch_size=32, verbose=0)
kfold = KFold(n_splits=10, shuffle=True, random_state=42)

In [39]:
#VALIACAO BASELINE
results = cross_val_score(estimator, X_data, dummy_y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Baseline: 8.31% (7.08%)
